<a href="https://colab.research.google.com/github/LeoMcBills/tensorFlowPrac/blob/main/legendZoro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary pytorch modules

In [ ]:
import torch
from torch import nn

# Download text-tinyShakespeare

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-21 12:38:04--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-03-21 12:38:04 (22.3 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
!ls

input.txt  sample_data


In [ ]:
!rm input.txt.1

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as text:
  text = text.read()

In [ ]:
print(len(text))
print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [ ]:
chars = sorted(list(set(text)))
print(len(chars))
print("".join(chars))

65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [ ]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
vocab_size = len(chars)

In [ ]:
print(vocab_size)
print(stoi)
print(itos)

65
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: '

In [ ]:
# method 1
def tokenize(word):
  newrd = []
  for piece in word:
    newrd.append(stoi[piece])
  return newrd

In [ ]:
# method 2
encode = lambda wrd: [stoi[char] for char in wrd]

In [ ]:
encoded = tokenize("Leo is the greatest programmer living")
encoded2 = encode("Leo is the greatest programmer living")

In [ ]:
print(encoded)
print(encoded2)

[24, 43, 53, 1, 47, 57, 1, 58, 46, 43, 1, 45, 56, 43, 39, 58, 43, 57, 58, 1, 54, 56, 53, 45, 56, 39, 51, 51, 43, 56, 1, 50, 47, 60, 47, 52, 45]
[24, 43, 53, 1, 47, 57, 1, 58, 46, 43, 1, 45, 56, 43, 39, 58, 43, 57, 58, 1, 54, 56, 53, 45, 56, 39, 51, 51, 43, 56, 1, 50, 47, 60, 47, 52, 45]


In [ ]:
# method 1
def wordFromToken(tokens):
  sentence = ""
  for token in tokens:
    sentence += itos[token]
  return sentence

In [ ]:
print(wordFromToken(encoded))

Leo is the greatest programmer living


In [ ]:
# method 2
decode = lambda wrd: [itos[num] for num in wrd]

In [ ]:
print("".join(decode(encoded)))

Leo is the greatest programmer living


# Get the training and test data

In [ ]:
train = tokenize(text[:int(len(text)*0.9)])
test = tokenize(text[int(len(text)*0.9):])

In [ ]:
print(train[:1000])
print(test[:1000])

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50, 10, 0, 31, 54, 43, 39, 49, 6, 1, 57, 54, 43, 39, 49, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 37, 53, 59, 1, 39, 56, 43, 1, 39, 50, 50, 1, 56, 43, 57, 53, 50, 60, 43, 42, 1, 56, 39, 58, 46, 43, 56, 1, 58, 53, 1, 42, 47, 43, 1, 58, 46, 39, 52, 1, 58, 53, 1, 44, 39, 51, 47, 57, 46, 12, 0, 0, 13, 50, 50, 10, 0, 30, 43, 57, 53, 50, 60, 43, 42, 8, 1, 56, 43, 57, 53, 50, 60, 43, 42, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 18, 47, 56, 57, 58, 6, 1, 63, 53, 59, 1, 49, 52, 53, 61, 1, 15, 39, 47, 59, 57, 1, 25, 39, 56, 41, 47, 59, 57, 1, 47, 57, 1, 41, 46, 47, 43, 44, 1, 43, 52, 43, 51, 63, 1, 58, 53, 1, 58, 46, 43, 1, 54, 43, 53, 54, 50, 43, 8, 0, 0, 13, 50, 50, 10, 0, 35, 43, 1, 49, 52, 53, 61, 5, 

## create input tensors

In [ ]:
train = torch.tensor(train, dtype=torch.long)
test = torch.tensor(test, dtype=torch.long)

<ipython-input-83-f51e0764ed65>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train = torch.tensor(train, dtype=torch.long)
<ipython-input-83-f51e0764ed65>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test = torch.tensor(test, dtype=torch.long)


In [ ]:
print(train)
print(test)

tensor([18, 47, 56,  ..., 43, 56, 43])
tensor([12,  0,  0,  ..., 45,  8,  0])


In [ ]:
batch_size = 8
block_size = 4

In [ ]:
print(train[:batch_size+1])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])


# Build a simple Bigram model

In [ ]:
class NeuralNet(nn.Module):
  pass